**Mount Google Drive locally**

Mount your Google Drive on your runtime using an authorization code



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import os
import cv2
import numpy as np
import keras
import numpy as np
import random
from imutils import paths
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from keras.applications import vgg16
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers

dataset_path = "/content/drive/My Drive/DATASETS/dataset"
IMAGE_DIMS = (224, 224, 3)
BS = 32
EPOCHS=100
print("keras version %s"%keras.__version__)
print("opencv version %s"%cv2.__version__)

# initialize the data and labels
data = []
labels = []

keras version 2.2.5
opencv version 4.1.2


In [5]:
# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images(dataset_path)))
random.seed(42)
random.shuffle(imagePaths)
# loop over the input images
for ind, imagePath in enumerate(imagePaths):
	# load the image, pre-process it, and store it in the data list
	print(ind, imagePath, imagePath.split(os.path.sep)[-2])
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = img_to_array(image)
	data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)

[INFO] loading images...
0 /content/drive/My Drive/DATASETS/dataset/bulbasaur/00000168.jpg bulbasaur
1 /content/drive/My Drive/DATASETS/dataset/pikachu/00000098.jpg pikachu
2 /content/drive/My Drive/DATASETS/dataset/mewtwo/00000090.jpg mewtwo
3 /content/drive/My Drive/DATASETS/dataset/squirtle/00000028.jpg squirtle
4 /content/drive/My Drive/DATASETS/dataset/mewtwo/00000150.jpg mewtwo
5 /content/drive/My Drive/DATASETS/dataset/charmander/00000218.jpg charmander
6 /content/drive/My Drive/DATASETS/dataset/bulbasaur/00000100.jpg bulbasaur
7 /content/drive/My Drive/DATASETS/dataset/pikachu/00000169.png pikachu
8 /content/drive/My Drive/DATASETS/dataset/mewtwo/00000019.jpg mewtwo
9 /content/drive/My Drive/DATASETS/dataset/squirtle/00000205.jpg squirtle
10 /content/drive/My Drive/DATASETS/dataset/charmander/00000075.png charmander
11 /content/drive/My Drive/DATASETS/dataset/charmander/00000210.png charmander
12 /content/drive/My Drive/DATASETS/dataset/squirtle/00000123.jpg squirtle
13 /conten

In [6]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {:.2f}MB".format(
	data.nbytes / (1024 * 1000.0)))

[INFO] data matrix: 1355.93MB


In [0]:
# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [0]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.2, random_state=42)

In [9]:
# initialize the model
print("[INFO] compiling model...")

# Load VGG16 model without the top layers
base_layers = vgg16.VGG16(include_top=False, input_shape=IMAGE_DIMS)
#base_layers = vgg16.VGG16(include_top=False)

# Allow fine tuning to go into the convolution layers
for layer in base_layers.layers:
     layer.trainable = False

# Create the network
network = models.Sequential(base_layers.layers)
network.add(layers.Flatten())
network.add(layers.Dense(5, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.summary()

[INFO] compiling model...




58892288/58889256 [==============================] - 2s 0us/step








Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________

In [10]:
history = network.fit(trainX, trainY, epochs=50, batch_size=32, verbose=2)
test_loss, test_acc = network.evaluate(testX, testY, verbose=2)
print('test_acc:', test_acc)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
 - 16s - loss: 4.5720 - acc: 0.5043
Epoch 2/50
 - 4s - loss: 3.6654 - acc: 0.6735
Epoch 3/50
 - 4s - loss: 3.6047 - acc: 0.7126
Epoch 4/50
 - 4s - loss: 3.5071 - acc: 0.7440
Epoch 5/50
 - 4s - loss: 3.5464 - acc: 0.7440
Epoch 6/50
 - 4s - loss: 3.4951 - acc: 0.7527
Epoch 7/50
 - 4s - loss: 3.4282 - acc: 0.7733
Epoch 8/50
 - 4s - loss: 3.4711 - acc: 0.7646
Epoch 9/50
 - 4s - loss: 3.4392 - acc: 0.7679
Epoch 10/50
 - 4s - loss: 3.4642 - acc: 0.7733
Epoch 11/50
 - 4s - loss: 3.4632 - acc: 0.7690
Epoch 12/50
 - 4s - loss: 3.4503 - acc: 0.7722
Epoch 13/50
 - 4s - loss: 3.4411 - acc: 0.7657
Epoch 14/50
 - 4s - loss: 3.4435 - acc: 0.7733
Epoch 15/50
 - 4s - loss: 3.4487 - acc: 0.7711
Epoch 16/50
 - 4s - loss: 3.4336 - acc: 0.7722
Epoch 17/50
 - 4s - loss: 3.4365 - acc: 0.7755
Epoch 18/50
 - 4s - loss: 3.4482 - acc: 0.7755
Epoch 19/50
 - 5s - loss: 3.4343 - acc: 0.7755
Epoch 20/50
 - 5s -

Load an image for testing

In [0]:
# load the image
#image = cv2.imread(dataset_path + '/mewtwo/00000000.jpg')
image = cv2.imread(dataset_path + '/pikachu/00000000.jpg')

# pre-process the image for classification
image = cv2.resize(image, (IMAGE_DIMS[0], IMAGE_DIMS[1]))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

In [14]:
# classify the input image
print("[INFO] classifying image...")
proba = network.predict(image)[0]
idx = np.argmax(proba)
label = lb.classes_[idx]

print(label)

[INFO] classifying image...
pikachu
